In [1]:
# Import python modules
import geopandas as gpd
import pandas as pd
import pyproj
import numpy as np
import fiona
import time
from geojson import Feature, Point, FeatureCollection
from shapely.geometry import shape, mapping
import scipy.spatial
import json

import datetime

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
##coordinates 
## Angola -> 32734
## Botswana -> 32735
## Burundi -> 32735
## DRC -> 32734
## Kenya -> 32736
## Madagascar -> 32739
## Malawi -> 32736
## Mozambique -> 32736
## Namibia -> 32734
## South_Africa -> 32734
## Tanzania -> 32735
## Uganda -> 32735
## Zambia -> 32735
## Zimbabwe -> 32736

In [3]:
cntry = "Zimbabwe"
cntryc = "zw"
cntryiso = "ZWE"

In [4]:
## Coordinate and projection systems
crs_WGS84 = pyproj.CRS("EPSG:4326")    # Originan WGS84 coordinate system
crs_proj = pyproj.CRS("EPSG:32736")    # Projection system for the selected country -- see http://epsg.io/ for more info

In [5]:
# Define path and name of the file
admin_path = r"C:\Users\alexl\Dropbox\Self-employment\Imperial work\Mineral Work\GIS_data\{}".format(cntry)
admin_name = "{}_admin0.gpkg".format(cntry)
admin_gdf = gpd.read_file(admin_path + "\\" + admin_name)

# Define path and name of the file
mine_path = r"C:\Users\alexl\Dropbox\Self-employment\Imperial work\Mineral Work\GIS_data\MiningData\DataFromRaghavJune2024"
mine_name = "optimal_locations_for_processing.gpkg"
lay_name = "2030_75_max_threshold_metal_tons"
mine_gdf = gpd.read_file(mine_path + "\\" + mine_name, layer=lay_name)
mine_gdf = mine_gdf[mine_gdf['iso3'] == cntryiso]

# Define path and name of the file
HV_path = r"C:\Users\alexl\Dropbox\Self-employment\Imperial work\Mineral Work\GIS_data\{}".format(cntry)
HV_name = "HV_lines.gpkg"
HV_gdf = gpd.read_file(HV_path + "\\" + HV_name)

# Path of result files
outpath= r"C:\Users\alexl\Dropbox\Self-employment\Imperial work\Mineral Work\Output_folder"

In [6]:
def processing_lines_bulk(lines, name, admin, crs, workspace, mines):
    #lines=gpd.read_file(file_name)

    lines_clip = gpd.clip(lines, admin)
    lines_clip.crs = {'init' :'epsg:4326'}
    lines_proj=lines_clip.to_crs({ 'init': crs})

    lines_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    line = fiona.open(workspace +  r"\ " + name + "_proj.shp")
    firstline = line.next()

    schema = {'geometry' : 'Point', 'properties' : {'id' : 'int'},}
    with fiona.open(workspace + r"\ " + name + "_proj_points.shp", "w", "ESRI Shapefile", schema) as output:
        for lines in line:
            if lines["geometry"] is not None:
                first = shape(lines['geometry'])
                length = first.length
                for distance in range(0,int(length),100):
                    point = first.interpolate(distance)
                    output.write({'geometry' :mapping(point), 'properties' : {'id':1}})

    lines_f = fiona.open(workspace + r"\ " + name + "_proj_points.shp")
    lines = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points = fiona.open(workspace + r'\mines_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in lines_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    mines[name+'Dist'] = z
    mines[name+'Dist'] = mines[name+'Dist']/1000   

    a = vector_overlap(lines, mines, name+'Dist')

    mines = pd.merge(left = mines, right = a[['id',name+'Dist2']], on='id', how = 'left')
    mines.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    mines.loc[mines[name+'Dist2'] == 0, name+'Dist'] = 0

    del mines[name+'Dist2']
    mines.rename(columns={name+'Dist': name+'Dist_km'}, inplace=True)
    
    print(datetime.datetime.now())
    return mines

In [10]:
mine_gdf = processing_lines_bulk(HV_gdf, "HV", admin_gdf, crs_proj, outpath, mine_gdf)

2024-07-03 13:19:45.161629


In [11]:
mine_gdf

,id,iso3,graphite_conversion_location_in_country,graphite_final_stage_production_tons_3.0_origin_in_country,graphite_gcosts_3.0_origin_in_country,graphite_distance_km_3.0_origin_in_country,graphite_time_hr_3.0_origin_in_country,lithium_conversion_location_in_country,lithium_final_stage_production_tons_3.0_origin_in_country,lithium_gcosts_3.0_origin_in_country,...,copper_final_stage_production_tons_1.0_origin_in_country,copper_gcosts_1.0_origin_in_country,copper_distance_km_1.0_origin_in_country,copper_time_hr_1.0_origin_in_country,distance_to_grid_meters,on_grid,nickel_final_stage_production_tons_1.0_origin_in_country,nickel_final_stage_production_tons_1.0_in_region,geometry,HVDist_km
0,city_1858,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,4318.578416,0,0.000000,0.000000,POINT (28.58330 -20.15000),4.065076
1,city_1860,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,3500.015925,0,0.000000,0.000000,POINT (31.05390 -17.82940),3.333554
2,node_14603468,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,4437.150732,0,0.000000,0.000000,POINT (30.37091 -18.18778),4.198096
3,node_14606785,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,47.184085,1,0.000000,0.000000,POINT (30.05247 -20.31452),0.040022
4,node_14649815,ZWE,0.0,0.0,0.000000,0.000000,0.000000,1.0,2836.291087,44.117946,...,0.0,0.0,0.0,0.0,55388.429106,0,0.000000,0.000000,POINT (31.45343 -20.40871),51.903110
5,railn_555014685,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,19264.554928,0,0.000000,0.000000,POINT (30.74524 -21.25336),17.961606
6,railn_555027634,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,8872.656682,0,0.000000,0.000000,POINT (29.87385 -20.29665),8.332092
7,railn_555063796,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,200.430517,1,0.000000,0.000000,POINT (29.80926 -19.45399),0.192690
8,railn_555064270,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,3736.885594,0,0.000000,0.000000,POINT (31.05229 -17.83496),3.559104
9,railn_555064677,ZWE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,8023.583940,0,0.000000,0.000000,POINT (30.01988 -22.18620),7.439224
